In [ ]:
import pandas as pd

from get_game_information import team_info, fetch_games, get_game_info
from game_simulator import outcomes, calculate_total_bases, simulator
from visualizations import run_dist, tb_barplot, la_ev_graph

teams_df = team_info()

## How far back you want to go for games
days_ago = 0.55

games_df, games_list, venues_list = fetch_games(days_ago)
## Add short team names
games_df = pd.merge(games_df, teams_df[['team.id', 'teamName']], left_on='teams.away.team.id', right_on='team.id')
games_df = games_df.rename(columns={'teamName': 'away.team'})
games_df = pd.merge(games_df, teams_df[['team.id', 'teamName']], left_on='teams.home.team.id', right_on='team.id')
games_df = games_df.rename(columns={'teamName': 'home.team'})
games_df = games_df.drop(columns=['team.id_x', 'team.id_y'])

game_info_list = []

for game_id, venue in zip(games_list, venues_list):
  game_data = get_game_info(game_id)
  if game_data is None:
      print(f"Error retrieving data for game ID: {game_id}")
      continue
  game_data['venue.name'] = venue
  ## Get actual game info
  game_info = games_df.copy()
  game_info = game_info[game_info['gamePk'] == game_id]

  home_team = game_info['home.team'].values[0]
  away_team = game_info['away.team'].values[0]
  home_score = int(game_info['teams.home.score'].iloc[0])
  away_score = int(game_info['teams.away.score'].iloc[0])

  home_outcomes = outcomes(game_data, 'home')
  away_outcomes = outcomes(game_data, 'away')

  # Calculate the estimated total bases
  home_estimated_total_bases = calculate_total_bases(home_outcomes)
  away_estimated_total_bases = calculate_total_bases(away_outcomes)

  num_simulations = 3500
  home_runs_scored, away_runs_scored, home_win_percentage, away_win_percentage, tie_percentage = simulator(num_simulations, home_outcomes, away_outcomes)
  run_dist(num_simulations, home_runs_scored, away_runs_scored, home_team, away_team, home_score, away_score, home_win_percentage, away_win_percentage, tie_percentage)
  # tb_barplot(home_estimated_total_bases, away_estimated_total_bases, home_win_percentage, away_win_percentage, tie_percentage, home_team, away_team, home_score, away_score)
  la_ev_graph(home_outcomes, away_outcomes, home_runs_scored, away_runs_scored, home_team, away_team, home_score, away_score, home_win_percentage, away_win_percentage, tie_percentage)